# Payoff d'un iron condor sur SPY

Iron condor (put spread + call spread en crédit) : payoff borné, P&L net (prime BS) et points morts.


In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
import sys
from pathlib import Path
from IPython.display import display, Markdown

_base = Path.cwd().resolve()
for extra in (_base, _base.parent, _base.parent.parent):
    candidate = extra / "notebooks" / "scripts"
    if candidate.exists() and str(candidate) not in sys.path:
        sys.path.insert(0, str(candidate))
        break
from pricing import fetch_spy_history, view_iron_condor, payoff_iron_condor
plt.style.use("seaborn-v0_8-darkgrid")

close_spy = fetch_spy_history()
spot_ref = float(close_spy.iloc[-1])

k_put_long = spot_ref * 0.9
k_put_short = spot_ref * 0.97
k_call_short = spot_ref * 1.03
k_call_long = spot_ref * 1.1
view_static = view_iron_condor(spot_ref, k_put_long, k_put_short, k_call_short, k_call_long)
premium = view_static["premium"]
credit = -premium
be_low, be_high = view_static["breakevens"]


## Évolution du sous-jacent (SPY)
Historique des clôtures sur un an et repère du spot de référence (dernier close).


In [ ]:

fig, ax = plt.subplots(figsize=(10, 4))
close_spy.plot(ax=ax, color="steelblue", label="SPY close")
ax.axhline(spot_ref, color="crimson", linestyle="--", label=f"Dernier close ≈ {spot_ref:.2f}")
ax.set_title("SPY - clôtures sur 1 an")
ax.set_xlabel("Date")
ax.set_ylabel("Prix")
ax.legend()
plt.show()


## Payoff de l'iron condor
Crédit initial, plateau central, pertes bornées sur les ailes. Affichage payoff, P&L net et points morts.


In [ ]:

    fig, ax = plt.subplots(figsize=(9, 5))
    ax.plot(view_static["s_grid"], view_static["payoff"], label="Payoff brut")
    ax.plot(view_static["s_grid"], view_static["pnl"], label="P&L net", color="darkorange")
    for k, style, label in [
        (k_put_long, ':', f'K put long = {k_put_long:.2f}'),
        (k_put_short, '--', f'K put short = {k_put_short:.2f}'),
        (k_call_short, '--', f'K call short = {k_call_short:.2f}'),
        (k_call_long, ':', f'K call long = {k_call_long:.2f}'),
    ]:
        ax.axvline(k, color='gray', linestyle=style, label=label)
    ax.axvline(be_low, color="forestgreen", linestyle=":", label=f"Point mort bas ≈ {be_low:.2f}")
    ax.axvline(be_high, color="forestgreen", linestyle="--", label=f"Point mort haut ≈ {be_high:.2f}")
    ax.axvline(spot_ref, color="crimson", linestyle="-.", label=f"S_0 = {spot_ref:.2f}")
    ax.axhline(0, color="black", linewidth=0.8)
    ax.set_xlabel("Spot")
    ax.set_ylabel("Payoff / P&L")
    ax.set_title("Iron condor : payoff et P&L (prime BS)")
    ax.legend(loc="upper left")
    plt.show()

    display(Markdown(f"""**Crédit net (BS) ≈ {credit:.4f}**

- Points morts : {be_low:.2f} / {be_high:.2f}
- Crédit = -prime (signe négatif si prime payée)."""))


## Payoff interactif
Curseurs pour S_0, S_T et les quatre strikes, crédit et points morts recalculés automatiquement.


In [ ]:

    spot0_slider = widgets.FloatSlider(value=spot_ref, min=spot_ref*0.5, max=spot_ref*1.5, step=1.0, description='S_0')
    spotT_slider = widgets.FloatSlider(value=spot_ref, min=spot_ref*0.5, max=spot_ref*1.5, step=1.0, description='S_T')
    slider_put_long = widgets.FloatSlider(value=k_put_long, min=spot_ref*0.4, max=spot_ref*1.0, step=1.0, description='K put long')
    slider_put_short = widgets.FloatSlider(value=k_put_short, min=spot_ref*0.6, max=spot_ref*1.1, step=1.0, description='K put short')
    slider_call_short = widgets.FloatSlider(value=k_call_short, min=spot_ref*0.9, max=spot_ref*1.4, step=1.0, description='K call short')
    slider_call_long = widgets.FloatSlider(value=k_call_long, min=spot_ref*1.0, max=spot_ref*1.6, step=1.0, description='K call long')
    output = widgets.Output()

    def _update_payoff(change=None):
        with output:
            output.clear_output()
            s0 = spot0_slider.value
            sT = spotT_slider.value
            k_put_long_val = slider_put_long.value
            k_put_short_val = slider_put_short.value
            k_call_short_val = slider_call_short.value
            k_call_long_val = slider_call_long.value

            view_dyn = view_iron_condor(s0, k_put_long_val, k_put_short_val, k_call_short_val, k_call_long_val)
            premium_dyn = view_dyn["premium"]
            credit_dyn = -premium_dyn
            be_low, be_high = view_dyn["breakevens"]
            payoff_T = float(payoff_iron_condor(sT, k_put_long_val, k_put_short_val, k_call_short_val, k_call_long_val))
            pnl_T = payoff_T - premium_dyn

            fig, ax = plt.subplots(figsize=(7, 4))
            ax.plot(view_dyn["s_grid"], view_dyn["payoff"], label='Payoff brut')
            ax.plot(view_dyn["s_grid"], view_dyn["pnl"], label='P&L net', color='darkorange')
            for k, style, label in [
                (k_put_long_val, ':', f'K put long = {k_put_long_val:.2f}'),
                (k_put_short_val, '--', f'K put short = {k_put_short_val:.2f}'),
                (k_call_short_val, '--', f'K call short = {k_call_short_val:.2f}'),
                (k_call_long_val, ':', f'K call long = {k_call_long_val:.2f}'),
            ]:
                ax.axvline(k, color='gray', linestyle=style, label=label)
            ax.axvline(be_low, color='forestgreen', linestyle=':', label=f'Point mort bas ≈ {be_low:.2f}')
            ax.axvline(be_high, color='forestgreen', linestyle='--', label=f'Point mort haut ≈ {be_high:.2f}')
            ax.axvline(s0, color='crimson', linestyle='-.', label=f'S_0 = {s0:.2f}')
            ax.axhline(0, color='black', linewidth=0.8)
            ax.scatter([sT], [pnl_T], color='gold', zorder=5, label=f'P&L à S_T = {pnl_T:.2f}')
            ax.set_xlabel('Spot')
            ax.set_ylabel('Payoff / P&L')
            ax.legend(loc='lower right')
            ax.set_title("Payoff et P&L de l'iron condor")
            plt.show()

            display(Markdown(f"""**S_0 = {s0:.2f}**, **S_T = {sT:.2f}**

- K put long = {k_put_long_val:.2f}
- K put short = {k_put_short_val:.2f}
- K call short = {k_call_short_val:.2f}
- K call long = {k_call_long_val:.2f}
- Crédit net (BS) = {credit_dyn:.4f}
- Payoff à l'expiration = {payoff_T:.4f}
- P&L net = {pnl_T:.4f}
- Points morts ≈ {be_low:.2f} / {be_high:.2f}"""))

    for sl in (spot0_slider, spotT_slider, slider_put_long, slider_put_short, slider_call_short, slider_call_long):
        sl.observe(_update_payoff, names='value')

    _update_payoff()
    display(widgets.VBox([spot0_slider, spotT_slider, slider_put_long, slider_put_short, slider_call_short, slider_call_long, output]))
